# Energy Explorary Analysis

In [1]:
import numpy as np
import pandas as pd
import os
import pathlib
        
import requests
import json

from scipy.stats import pearsonr,spearmanr, boxcox
from scipy import stats

from statsmodels.graphics.gofplots import qqplot

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import seaborn as sns

import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.figure_factory as ff

from IPython.display import display, Image

# pio.renderers.default = 'browser'
pio.renderers.default = 'notebook'

import dash
from dash.dependencies import Input, Output, State
# import dash_core_components as dcc
from dash import dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

from jupyter_dash import JupyterDash

import warnings
warnings.filterwarnings('ignore')

# style.use('fivethirtyeight')

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

rand_state=1000

C:\Users\zengx\AppData\Local\Temp/ipykernel_19112/4003285519.py:34: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [2]:
# numeric columns
def numeric_columns(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','datetime64[ns]']
    df_numeric = df.select_dtypes(include=numerics)
    return df_numeric.columns.tolist()

# object columns
def object_columns(df):
    objects = ['object']
    df_object = df.select_dtypes(include=objects)
    return df_object.columns.tolist()

# change column data type to categorical
def cat_features(df, ls):
    for l in ls:
        df[l]=df[l].astype(str)
    return df

# change column data type to float
def num_features(df, ls):
    for l in ls:
        df[l]=df[l].astype(float)
    return df

def missing_percentage(df, title):
    nadf=(round(df.isna().sum().sort_values(ascending=True)/len(df),4)).reset_index()
    nadf.columns=['Variable','Missing_percentage']
    nadf=nadf[nadf['Missing_percentage']>0].sort_values(by='Missing_percentage', ascending=False)

    if len(nadf)>0 :        
        fig, ax = plt.subplots(figsize=(8,3))
        sns.barplot(x='Missing_percentage',
               y='Variable',
               data=nadf.head(50),
               palette='Spectral').set(title=title)
    
    return nadf

## Import Data from CSV Generated in The Last Step

In [3]:
app_path=str(pathlib.Path().parent.resolve())

energy=pd.read_csv(os.path.join(app_path, 'energy.csv'))


In [4]:
i1='Source EUI (kBtu/ft²)'
i2='Site EUI (kBtu/ft²)'
i3='Fuel Oil Use Total Intensity (kBtu/ft2)'
i4='Natural Gas Use Intensity (kBtu/ft2)'
i5='Diesel #2 Use Intensity (kBtu/ft2)'
i6='District Chilled Water Use Intensity (kBtu/ft2)'
i7='District Steam Use Intensity (kBtu/ft2)'
i8='Green Power - Onsite and Offsite Intensity (kBtu/ft2)'
i9='Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems Intensity (kBtu/ft2)'

in9='Electricity Use Intensity (kBtu/ft2)'

t1='Source Energy Use (kBtu)'
t2='Site Energy Use (kBtu)'
t3='Fuel Oil Use Total (kBtu)'
t4='Natural Gas Use (kBtu)'
t5='Diesel #2 Use (kBtu)'
t6='District Chilled Water Use (kBtu)'
t7='District Steam Use (kBtu)'
t8='Green Power - Onsite and Offsite (kBtu)'
t9='Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu)'

tn9='Electricity Use (kBtu)'

e1='Total GHG Emissions Intensity (kgCO2e/ft²)'
e2='Indirect GHG Emissions Intensity (kgCO2e/ft²)'
e3='Direct GHG Emissions Intensity (kgCO2e/ft²)'
e4='Avoided Emissions - Onsite and Offsite Green Power Intensity (Metric Tons CO2e/ft2)'

en4='Avoided Emissions Intensity (Metric Tons CO2e/ft2)'

s1='Total GHG Emissions (Metric Tons CO2e)'
s2='Indirect GHG Emissions (Metric Tons CO2e)'
s3='Direct GHG Emissions (Metric Tons CO2e)'
s4='Avoided Emissions - Onsite and Offsite Green Power (Metric Tons CO2e)'

sn4='Avoided Emissions Intensity (Metric Tons CO2e/ft2)'


In [5]:
emission_dict_intensity={e1:'Total GHG Emissions',
    e2:'Indirect GHG Emissions',
    e3:'Direct GHG Emissions',
    e4:'Avoided Emissions'
}

emission_dict={s1:'Total GHG Emissions',
    s2:'Indirect GHG Emissions',
    s3:'Direct GHG Emissions',
    s4:'Avoided Emissions'
}

In [6]:
use_dict={t1:'Source Energy', 
           t2:'Site Energy',
           t3:'Fuel Oil', 
           t4:'Natural Gas',
           t5:"Diesel #2", 
           t6:'Chilled Water',
           t7:'District Steam',
           t8:'Green Power',
           t9:'Electricity'}

use_dict_intensity={i1:'Source Energy', 
           i2:'Site Energy',
           i3:'Fuel Oil', 
           i4:'Natural Gas',
           i5:"Diesel #2", 
           i6:'Chilled Water',
           i7:'District Steam',
           i8:'Green Power',
           i9:'Electricity'}

In [7]:
def remove_small_outliers(df, f1):
    df[f1]=np.where(df[f1]<1, np.nan, df[f1])
    return df

energy=remove_small_outliers(energy, i1)
energy=remove_small_outliers(energy, i2)
energy=remove_small_outliers(energy, i3)
energy=remove_small_outliers(energy, i4)
energy=remove_small_outliers(energy, i5)
energy=remove_small_outliers(energy, i6)
energy=remove_small_outliers(energy, i7)
energy=remove_small_outliers(energy, i8)
energy=remove_small_outliers(energy, i9)

energy=remove_small_outliers(energy, e1)
energy=remove_small_outliers(energy, e2)
energy=remove_small_outliers(energy, e3)
energy=remove_small_outliers(energy, e4)

In [8]:
l_head=['Property Id',
    'Property Name',
    'bbl',
    'Address 1',
    'City',
    'Postcode',
    'Primary Property Type - Self Selected',
    'Borough',
    'Council District',
    'Census Tract',
    'isLowrise',
    'Latitude',
    'Longitude',
    'Year Built',
    'Community Board',
    'NTA']   

energy[l_head]=energy[l_head].fillna(0)
    
df_total=energy.groupby(l_head).sum().reset_index()
df_total['Year Ending']=2015
df_mean=energy.groupby(l_head).mean().reset_index()
df_mean['Year Ending']=2014

df_energy=pd.concat([energy, df_total, df_mean], ignore_index=True)
df_energy['Community Board']=df_energy['Community Board'].round().astype(int).astype(str)
df_energy['ENERGY STAR Score'].fillna(0,inplace=True)
df_sample=df_energy.sample(frac=0.1, random_state=rand_state)

In [9]:
df_energy['Total GHG Emissions Intensity (kgCO2e/ft²)']

0       4.000
1       3.500
2         NaN
3         NaN
4         NaN
         ... 
45779     NaN
45780     NaN
45781   4.100
45782   1.100
45783   5.300
Name: Total GHG Emissions Intensity (kgCO2e/ft²), Length: 45784, dtype: float64

In [10]:
Cat_list=['Council District',
    'isLowrise',
    'Community Board',
    'NTA',
    'Borough']
df_use=df_energy[['Property Id', 'Property Name', 
                   'Year Ending',
                   i1,i2,i3,i4,i5,i6,i7,i8,i9,
                   t1,t2,t3,t4,t5,t6,t7,t8,t9]+Cat_list]
df_use=df_use[~df_use[i1].isna()]

df_emission=df_energy[['Property Id', 'Property Name', 
                   'Year Ending',
                   e1, e2,e3,e4,
                   s1,s2,s3,s4]+Cat_list]
df_emission=df_emission[~df_emission[s1].isna()]

In [11]:
# the style arguments for the sidebar.
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '20%',
    'padding': '20px 10px',
    'background-color': '#f8f9fa'
}

# the style arguments for the main content page.
CONTENT_STYLE = {
    'margin-left': '25%',
    'margin-right': '5%',
    'top': 0,
    'padding': '20px 10px'
}

TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#191970'
}

CARD_TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#0074D9'
}


TAB_STYLE = {
    'width': 'inherit',
    'border': 'none',
    'boxShadow': 'inset 0px -1px 0px 0px lightgrey',
    'background': 'white',
    'paddingTop': 0,
    'paddingBottom': 0,
    'height': '42px',
}

SELECTED_STYLE = {
    'width': 'inherit',
    'boxShadow': 'none',
    'borderLeft': 'none',
    'borderRight': 'none',
    'borderTop': 'none',
    'borderBottom': '2px #004A96 solid',
    'background': 'white',
    'paddingTop': 0,
    'paddingBottom': 0,
    'height': '42px',
}

HR_STYLE = {
    'overflow': 'visible',
    'padding': 0,
    'border': 'none',
    'border-top': 'medium double #333',
    'color': '#333',
    'text-align': 'center',
}

GRAPH_STYLE = {
    "align" : "center",
    "justify" : "center",
    'horrizontal-align': 'center'
}


In [12]:
def drawrangeslider(id):
    return dcc.RangeSlider(min=2014, max=2020, step=1, 
                            value=[2020], id=id,
                           marks={
                            2016: {'label': '2016'},
                            2017: {'label': '2017'},
                            2018: {'label': '2018'},
                            2019: {'label': '2019'},
                            2020: {'label': '2020'},
                            2015:{'label': 'Total'},
                            2014:{'label': 'Mean'}}
                           )

def drawfigure(id):
    return dcc.Graph(id=id,
                    style=GRAPH_STYLE)

def drawradio(id):
    return dcc.RadioItems(        
                id=id,
                options=[{
                    'label': v,
                    'value': v
                } for v in Cat_list],
                value='Community Board',
                labelStyle={'display': 'block'}
    )

# Card
def drawcard(id):    
    return html.Div([
        html.H4("",id=id)
    ], style={'textAlign': 'center', 'color':'blue'})


# Text field
def drawtext(id):    
    return html.Div([
        html.H6("",id=id)
    ], style={'textAlign': 'center', 'color':'dark'})

def drawtable(id):
    return dash.dash_table.DataTable(
            id=id,
            page_current=0,
            page_size=10,
            page_action='custom',

            sort_action='custom',
            sort_mode='single',
            sort_by=[],
            style_table={'minWidth': '100%'},
            style_cell={
                # all three widths are needed
                'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(220, 220, 220)',
                }
            ],
            style_header={
                'backgroundColor': 'darkcyan',
                'color': 'black',
                'whiteSpace': 'normal',
                'height': 'auto',
            }
    )

def drawdropdown(id):
    return dcc.Dropdown(df_energy['Community Board'].unique(), value='103',
                        id=id)

In [13]:
controls = dbc.Col(
    [
        dbc.Row([
            html.P('Data Range', style={
                'textAlign': 'center'
            }),
            drawrangeslider(id='Year'),
        ]),
        dbc.Row([
            html.Hr(style=HR_STYLE)
        ]),
        dbc.Row([
            html.P('Categories', style={
                'textAlign': 'center'
            }),        
            drawradio(id='Categories'),
        ])
    ]
)

sidebar = html.Div(
    [
        html.H2('Please select ...', style=TEXT_STYLE),
        html.Hr(),
        controls
    ],
    style=SIDEBAR_STYLE,
)

In [26]:
content1_first_row = dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_property'),
                        drawtext(id='t_property')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_usage'),
                        drawtext(id='t_usage')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_emission'),
                        drawtext(id='t_emission')
                    ]
                )
            ]
        ),
        md=4
    )
])

# Following is the third row with one column with a figure.

content1_second_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='energymap'), md=12,
        )
    ]
)


content1_third_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='bar_category_use'), md=8
        ),
        dbc.Col(
            drawfigure(id='pie_category_use'), md=4
        )
    ], style= {'display': 'none'}
)


content1_fourth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='bar_category_emission'), md=8
        ),
        dbc.Col(
            drawfigure(id='pie_category_emission'), md=4
        )
    ], style= {'display': 'none'}
)


content1_fifth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_use'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_use'), md=6
        )
    ], style= {'display': 'none'}
)

content1_sixth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_emission'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_emission'), md=6
        )
    ], style= {'display': 'none'}
)

content1_seventh_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_use_by_category'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_use_by_category'), md=6
        )
    ], style= {'display': 'none'}
)

content1_eighth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_emission_by_category'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_emission_by_category'), md=6
        )
    ], style= {'display': 'none'}
)


content1_ninth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='bar_category_use_avg'), md=8
        ),
        dbc.Col(
            drawfigure(id='pie_category_use_avg'), md=4
        )
    ], style= {'display': 'none'}
)
# The following is the second row have 2 columns with figures.

content1_tenth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='bar_category_emission_avg'), md=8
        ),
        dbc.Col(
            drawfigure(id='pie_category_emission_avg'), md=4
        )
    ], style= {'display': 'none'}
)


content1_eleventh_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_use_avg'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_use_avg'), md=6
        )
    ]
)

content1_twelfth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_emission_avg'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_emission_avg'), md=6
        )
    ]
)

content1_thirteenth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_use_by_category_avg'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_use_by_category_avg'), md=6
        )
    ]
)

content1_fourteenth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_yeartrend_emission_by_category_avg'), md=6
        ),
        dbc.Col(
            drawfigure(id='bar_yeartrend_emission_by_category_avg'), md=6
        )
    ]
)


content1 = html.Div(
    [
        html.H2('Manhattan Energy Usage and Emission', id='title1', style=TEXT_STYLE),
        html.Hr(),
        content1_first_row,
        content1_second_row,
        content1_third_row,
        content1_fourth_row,
        content1_fifth_row,
        content1_sixth_row,
        content1_seventh_row,
        content1_eighth_row,
        content1_ninth_row,
        content1_tenth_row,
        content1_eleventh_row,
        content1_twelfth_row,
        content1_thirteenth_row,
        content1_fourteenth_row,
    ],
    style=CONTENT_STYLE
)


In [32]:
content2_first_row= dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_use_property'),
                        drawtext(id='t_use_property')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_use_total'),
                        drawtext(id='t_use_total')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_use_avg'),
                        drawtext(id='t_use_avg')
                    ]
                )
            ]
        ),
        md=4
    )
])

content2_second_row= dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        html.H6(id='Category_text'),
                        html.Div(id='dropdown_category',
                            children=[drawdropdown(id='category_list')]
                        )
                    ]
                )
            ]
        ),
        md=4
    ),
])

content2_third_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='usemap'), md=12,
        )
    ]
)

content2_fourth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_use_yeartrend_total'), md=6
        ),
        dbc.Col(
            drawfigure(id='pie_use_yeartrend_total'), md=6
        )
    ]
)

content2_fifth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_use_yeartrend_avg'), md=6, align='center'
        ),
        dbc.Col(
            drawfigure(id='pie_use_yeartrend_avg'), md=6, align='center'
        )
    ]
)

content2_sixth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_use_yeartrend_mean'), md=6, align="center"
        ),
        dbc.Col(
            drawfigure(id='pie_use_yeartrend_mean'), md=6, align="center"
        )
    ]
)

content2_seventh_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='heatmap_use'), md=12, align="center"
        )
    ], style= {'display': 'none'}
)

content2_eighth_row=dbc.Row(
    [
    ]
)
        
content2 = html.Div(
    [
        html.H2('Manhattan Energy Usage', id='title2', style=TEXT_STYLE),
        html.Hr(),
        content2_first_row,
        content2_second_row,
        content2_third_row,
        content2_fourth_row,
        content2_fifth_row,
        content2_sixth_row,
        content2_seventh_row,
        content2_eighth_row
    ],
    style=CONTENT_STYLE
)

In [33]:
content3_first_row= dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_emission_property'),
                        drawtext(id='t_emission_property')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_emission_total'),
                        drawtext(id='t_emission_total')
                    ]
                )
            ]
        ),
        md=4
    ),
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        drawcard(id='c_emission_avg'),
                        drawtext(id='t_emission_avg')
                    ]
                )
            ]
        ),
        md=4
    )
])

content3_second_row= dbc.Row([
    dbc.Col(
        dbc.Card(
            [

                dbc.CardBody(
                    [
                        html.H6(id='Category_text_emission'),
                        html.Div(id='dropdown_category_emission',
                            children=[drawdropdown(id='category_list_emission')]
                        )
                    ]
                )
            ]
        ),
        md=4
    ),
])

content3_third_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='emissionmap'), md=12,
        )
    ]
)

content3_fourth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_emission_yeartrend_total'), md=6
        ),
        dbc.Col(
            drawfigure(id='pie_emission_yeartrend_total'), md=6
        )
    ]
)

content3_fifth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_emission_yeartrend_avg'), md=6, align='center'
        ),
        dbc.Col(
            drawfigure(id='pie_emission_yeartrend_avg'), md=6, align='center'
        )
    ]
)

content3_sixth_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='line_emission_yeartrend_mean'), md=6, align="center"
        ),
        dbc.Col(
            drawfigure(id='pie_emission_yeartrend_mean'), md=6, align="center"
        )
    ]
)

content3_seventh_row = dbc.Row(
    [
        dbc.Col(
            drawfigure(id='heatmap_emission'), md=12, align="center"
        )
    ], style= {'display': 'none'}
)

content3_eighth_row=dbc.Row(
    [
    ]
)
        
content3 = html.Div(
    [
        html.H2('Manhattan Green Gas Emission', id='title3', style=TEXT_STYLE),
        html.Hr(),
        content3_first_row,
        content3_second_row,
        content3_third_row,
        content3_fourth_row,
        content3_fifth_row,
        content3_sixth_row,
        content3_seventh_row,
        content3_eighth_row
    ],
    style=CONTENT_STYLE
)

In [34]:
tabs= html.Div(
    [
        dcc.Tabs(id='tabs', className='row',value='overview',
                children=[
                    dcc.Tab(label='OverView', 
                            value='overview', 
                            children=content1,
                            style=TAB_STYLE,
                            selected_style=SELECTED_STYLE
                           ),
                    dcc.Tab(label='Energy Usage', 
                            value='usage', 
                            style=TAB_STYLE,
                            children=content2,
                            selected_style=SELECTED_STYLE
                           ),
                    dcc.Tab(label='Greenhouse Gas Emission', 
                            value='emission', 
                            style=TAB_STYLE,
                            children=content3,
                            selected_style=SELECTED_STYLE
                           ),
                    dcc.Tab(label='Tables', value='tables', 
                            style=TAB_STYLE,
                            selected_style=SELECTED_STYLE
                           )
                ],
                style=CONTENT_STYLE),
        
        html.Div(
            id='tabs-content',
            children=[])
    ]
)

In [35]:
app = JupyterDash(external_stylesheets=[dbc.themes.MATERIA],
                meta_tags=[{'name': 'viewport', 'content': 'width=device-width, ''initial-scale=1'}])
# app.config.suppress_callback_exceptions = True
app.title='Manhattan Energy Usage and Emission Analysis'

app.layout=html.Div([sidebar, tabs])

# @app.callback(Output('tabs-content', 'children'),
#               [Input('tabs', 'value')])
# def render_content(tab):
#     if tab == 'overview':
#         return content1
#     elif tab == 'usage':
#         return content2
#     elif tab == 'emission':
#         return content3
#     elif tab == 'tables':
#         return content4

@app.callback(
    Output('c_property', 'children'),
    Output('t_property', 'children'),
    Output('c_usage', 'children'),
    Output('t_usage', 'children'),
    Output('c_emission', 'children'),
    Output('t_emission', 'children'),
    [Input('Year', 'value'),
    Input('Categories', 'value')])
def update_cards(years, category):
    df=df_energy[df_energy['Year Ending'].isin(years)]
    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
    card1=str("{:,}".format(len(df['Property Id'].unique())))
    text1='total properties '+YearLabel
    
    card2=str("{:,}".format(df[t1].sum().round(4))) + ' (kBtu)'
    text2='total source energy used '+YearLabel
    
    card3=str("{:,}".format(df[s1].sum().round(4))) +' (kWh)'
    text3='total GHG emission '+YearLabel
    
    return [card1, text1, card2, text2, card3, text3]


@app.callback(
    Output('energymap', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value')])
def update_map(years, category):
          
    df=df_energy[df_energy['Year Ending'].isin(years)]
    lat=df[df['Community Board']=='103']['Latitude'].iloc[0]
    lon=df[df['Community Board']=='103']['Longitude'].iloc[0]
    fig1 = px.scatter_mapbox(
        df, 
        lat="Latitude", lon="Longitude", 
        color=category, 
        opacity=0.6,
        height=600,
        hover_name='Property Name', 
        hover_data=['ENERGY STAR Score',
                    'Site EUI (kBtu/ft²)',
                    'Source EUI (kBtu/ft²)',
                    'Total GHG Emissions Intensity (kgCO2e/ft²)'], 
                    size='ENERGY STAR Score',
                    size_max=15)
    
    fig1.update_layout(
                    mapbox=dict(
                        bearing=0,
                        center=dict(
                            lat=40.7729206,
                            lon=-73.9824891
                        ),
                        pitch=0,
                        zoom=12))
    fig1.update_layout(mapbox_style="open-street-map")
    fig1.update_layout(showlegend=False)
    fig1.update_layout(title='Energy Usage and Emission Map for Manhattan')

    return fig1

@app.callback(
    Output('bar_category_use', 'figure'),
    Output('pie_category_use', 'figure'),
    Output('bar_category_emission', 'figure'),
    Output('pie_category_emission', 'figure'),
    Output('line_yeartrend_use', 'figure'),
    Output('bar_yeartrend_use', 'figure'),
    Output('line_yeartrend_emission', 'figure'),
    Output('bar_yeartrend_emission', 'figure'),
    Output('line_yeartrend_use_by_category', 'figure'),
    Output('bar_yeartrend_use_by_category', 'figure'),
    Output('line_yeartrend_emission_by_category', 'figure'),
    Output('bar_yeartrend_emission_by_category', 'figure'),
    
    Output('bar_category_use_avg', 'figure'),
    Output('pie_category_use_avg', 'figure'),
    Output('bar_category_emission_avg', 'figure'),
    Output('pie_category_emission_avg', 'figure'),
    Output('line_yeartrend_use_avg', 'figure'),
    Output('bar_yeartrend_use_avg', 'figure'),
    Output('line_yeartrend_emission_avg', 'figure'),
    Output('bar_yeartrend_emission_avg', 'figure'),
    Output('line_yeartrend_use_by_category_avg', 'figure'),
    Output('bar_yeartrend_use_by_category_avg', 'figure'),
    Output('line_yeartrend_emission_by_category_avg', 'figure'),
    Output('bar_yeartrend_emission_by_category_avg', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value')])
def update_bar(years, category):    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
        
    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', category, t1, s1]]
    df1=df.groupby(category).sum().reset_index()
    
    fig1 = px.bar(df1, y=t1, x=category, color=category,
                  text=t1,
                  color_discrete_sequence=px.colors.sequential.Plasma_r,
                  title="Energy Usage for "+YearLabel)
#     fig1.update_layout(showlegend=False)
    fig1.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    fig1.update_layout(xaxis_showticklabels=False)
    fig1.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    
    
    fig2 = px.pie(df1, values=t1, names=category,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)
    fig2.update_layout(showlegend=False)
    
    
    fig3 = px.bar(df1, y=s1, x=category, color=category, 
                  text=s1,
                  color_discrete_sequence=px.colors.sequential.Plasma_r,
                  title="Greenhouse Gas Emission for "+YearLabel)
#     fig3.update_layout(showlegend=False)
    fig3.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    fig3.update_layout(xaxis_showticklabels=False)
    fig3.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    
    
    fig4 = px.pie(df1, values=s1, names=category,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)
    fig4.update_layout(showlegend=False)
    
    df_trend=df_energy[df_energy['Year Ending'].isin([2016,2017,2018,2019,2020])]
    df_trend=df_trend.groupby('Year Ending').sum().reset_index()
    fig5 = px.line(df_trend, x='Year Ending', y=t1, markers=True,
                  title='Total Energy Usage Trend for 2016 - 2020')     
    fig5.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
    fig5_1=px.bar(df_trend, y=t1, x='Year Ending', 
                  text=t1)
#     fig3.update_layout(showlegend=False)
    fig5_1.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    
    fig6 = px.line(df_trend, x='Year Ending', y=s1, markers=True,
                  title='Total Greenhouse Gas Emission Trend for 2016 - 2020')  
    fig6.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
    fig6_1=px.bar(df_trend, y=s1, x='Year Ending', 
                  text=s1)
#     fig3.update_layout(showlegend=False)
    fig6_1.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
        
    df_trend=df_energy[df_energy['Year Ending'].isin([2016,2017,2018,2019,2020])]
    df_trend=df_trend.groupby([category, 'Year Ending']).sum().reset_index()
    fig7 = px.line(df_trend, x='Year Ending', y=t1, color=category, markers=True,
                  title='Total Energy Usage Trend by ' + category +' for 2016 - 2020')     
    fig7.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
#     fig7_1 = px.bar(df_trend, x="Year Ending", y=t1, color=category, 
#                   text=t1)
#     fig7_1.update_layout(showlegend=False)

    fig7_1 = px.sunburst(df_trend, path=['Year Ending', category], values=t1, color=category)
    fig7_1.update_layout(showlegend=False)
    
    
    fig8 = px.line(df_trend, x='Year Ending', y=s1, 
                   color=category, markers=True,
                  title='Total Greenhouse Gas Emission Trend by '+category + ' for 2016 - 2020')  
    fig8.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
#     fig8_1 = px.bar(df_trend, x="Year Ending", y=s1, color=category,  
#                   text=s1)
#     fig8_1.update_layout(showlegend=False)
    fig8_1 = px.sunburst(df_trend, path=['Year Ending', category], values=s1, color=category)
    fig8_1.update_layout(showlegend=False)
    

    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', category, i1, e1]]
    df1=df.groupby(category).mean().reset_index()
    
    fig1_2 = px.bar(df1, y=i1, x=category, color=category,
                  text=i1,
                  color_discrete_sequence=px.colors.sequential.Plasma_r,
                  title="Average Energy Usage for "+YearLabel)
#     fig1.update_layout(showlegend=False)
    fig1_2.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    fig1_2.update_layout(xaxis_showticklabels=False)
    fig1_2.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    
    
    fig2_2 = px.pie(df1, values=i1, names=category,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)
    fig2_2.update_layout(showlegend=False)
    
    
    fig3_2 = px.bar(df1, y=e1, x=category, color=category, 
                  text=e1,
                  color_discrete_sequence=px.colors.sequential.Plasma_r,
                  title="Average Greenhouse Gas Emission for "+YearLabel)
#     fig3.update_layout(showlegend=False)
    fig3_2.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    fig3_2.update_layout(xaxis_showticklabels=False)
    fig3_2.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    
    
    fig4_2 = px.pie(df1, values=e1, names=category,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)
    fig4_2.update_layout(showlegend=False)
    

    
    df_trend=df_energy[df_energy['Year Ending'].isin([2016,2017,2018,2019,2020])]
    df_trend=df_trend.groupby('Year Ending').mean().reset_index()
    fig9 = px.line(df_trend, x='Year Ending', y=i1, markers=True,
                  title='Average Energy Usage Trend for 2016 - 2020')     
    fig9.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
    fig9_1=px.bar(df_trend, y=i1, x='Year Ending', 
                  text=i1)
#     fig3.update_layout(showlegend=False)
    fig9_1.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    
    fig10 = px.line(df_trend, x='Year Ending', y=e1, markers=True,
                  title='Average Greenhouse Gas Emission Trend for 2016 - 2020')  
    fig10.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
    fig10_1=px.bar(df_trend, y=e1, x='Year Ending', 
                  text=e1)
#     fig3.update_layout(showlegend=False)
    fig10_1.update_xaxes(tickangle=45,
                 tickmode = 'array'
                 )
    
    df_trend=df_energy[df_energy['Year Ending'].isin([2016,2017,2018,2019,2020])]
    df_trend=df_trend.groupby([category, 'Year Ending']).mean().reset_index()
    fig11 = px.line(df_trend, x='Year Ending', y=i1, color=category, markers=True,
                  title='Average Energy Use Intensity Trend by ' + category +' for 2016 - 2020')     
    fig11.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    
#     fig11_1 = px.bar(df_trend, x="Year Ending", y=i1, color=category, 
#                   text=i1)
#     fig11_1.update_layout(showlegend=False)

    fig11_1 = px.sunburst(df_trend, path=['Year Ending', category], values=i1, color=category)
    fig11_1.update_layout(showlegend=False)
    
    
    fig12 = px.line(df_trend, x='Year Ending', y=e1, 
                   color=category, markers=True,
                  title='Average GHG Emission Intensity Trend by '+category + ' for 2016 - 2020')  
    fig12.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
#     fig12_1 = px.bar(df_trend, x="Year Ending", y=e1, color=category,  
#                   text=e1)
#     fig12_1.update_layout(showlegend=False)
    fig12_1 = px.sunburst(df_trend, path=['Year Ending', category], values=e1, color=category)
    fig12_1.update_layout(showlegend=False)
    
    return [fig1,fig2, fig3, fig4,
            fig5, fig5_1, fig6, fig6_1, fig7, fig7_1, fig8, fig8_1, 
            fig1_2,fig2_2, fig3_2, fig4_2, 
            fig9, fig9_1, fig10, fig10_1, fig11, fig11_1, fig12, fig12_1]

## ENERGY USE CONTENT
@app.callback(
    Output('c_use_property', 'children'),
    Output('t_use_property', 'children'),
    Output('c_use_total', 'children'),
    Output('t_use_total', 'children'),
    Output('c_use_avg', 'children'),
    Output('t_use_avg', 'children'),
    Output('Category_text','children'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list', 'value')])
def update_cards(years, category, cat_value):
        
    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', category, 'Year Ending',
                                                       i1,i2,i3,i4,i5,i6,i7,i8,i9,
                                                       t1,t2,t3,t4,t5,t6,t7,t8,t9]]
    df=df[~df[t1].isna()]    
    df=df_energy[df_energy['Year Ending'].isin(years)]
    df=df[df[category]==cat_value]  
    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
    
    
    card1=str("{:,}".format(df[t1].count()))
    text1='Property Collected  '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    card2=str("{:,}".format(round(df[t1].sum(), 4))) + ' (kBtu)'
    text2='Total Source Energy Used '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    card3=str("{:,}".format(round(df[t1].mean(), 4))) +' (kWh)'
    text3='Average Source Energy Used '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    category_name=category+' : '
    
    return [card1, text1, card2, text2, card3, text3, category_name]
        


@app.callback(
    Output('dropdown_category', 'children'),
    [Input('Categories', 'value')])
def update_dropdown_category(category):
    value=df_energy[category].unique()[0]
    if category=='Community Board':
        value='103'
    elif category=='Borough':
        value='MANHATTAN'
    return dcc.Dropdown(df_energy[category].unique(), value=value,
                        id='category_list')

@app.callback(
    Output('usemap', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list', 'value')])
def update_map(years, category, cat_value):      
    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', 'Property Name', 
                                                        category, 'Year Ending',
                                                        'Latitude','Longitude',
                                                       i1,i2,i3,i4,i5,i6,i7,i8,i9,
                                                       t1,t2,t3,t4,t5,t6,t7,t8,t9]]
    df=df[~df[i1].isna()]
    df=df[df[category]==cat_value]  
    lat=df['Latitude'].iloc[0]
    lon=df['Longitude'].iloc[0]
    fig1 = px.scatter_mapbox(
        df, 
        lat="Latitude", 
        lon="Longitude", 
        color=category, 
        opacity=0.6,
        height=600,
        hover_name='Property Name', 
        hover_data=[t1,t2,t3,t4,t5,t6,t7,t8,t9],
                    size=t1,
                    size_max=15,
                    zoom=10)
    
    fig1.update_layout(
                    mapbox=dict(
                        bearing=0,
                        center=dict(
#                             lat=40.7729206,
#                             lon=-73.9824891
                            lat=lat,
                            lon=lon,
                        ),
                        pitch=0,
                        zoom=12))
    fig1.update_layout(mapbox_style="carto-positron")
    fig1.update_layout(showlegend=False)
    fig1.update_layout(title='Energy Usage Map for Manhattan')

    return fig1


@app.callback(
    Output('line_use_yeartrend_total', 'figure'),
    Output('pie_use_yeartrend_total', 'figure'),
    Output('line_use_yeartrend_avg', 'figure'),
    Output('pie_use_yeartrend_avg', 'figure'),
    Output('line_use_yeartrend_mean', 'figure'),
    Output('pie_use_yeartrend_mean', 'figure'),
    Output('heatmap_use', 'figure'),
#     Output('bar_use_total', 'figure'),
#     Output('pie_use_total', 'figure'),
#     Output('bar_use_mean', 'figure'),
#     Output('pie_use_mean', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list', 'value')])
def update_bar(years, category, cat_value):    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
        
    
    df_trend=df_use[df_use[category]==cat_value]  
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').sum().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[t1,t2,t3,t4,t5,t6,t7,t8,t9],
            var_name='Source Type',
            value_name='Value (kBtu)')   
    
    fig1 = px.line(df_trend, x='Year Ending', y='Value (kBtu)', markers=True,
                   color='Source Type',
                   title='Total Energy Usage Trend for 2016 - 2020 for '+cat_value,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)    
    fig1.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig1.for_each_trace(lambda t: t.update(name = use_dict[t.name],
                                      legendgroup = use_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, use_dict[t.name])
                                     )
                  )
    

    fig2 = px.sunburst(df_trend, path=['Year Ending', 'Source Type'], values='Value (kBtu)', color='Source Type',
                       color_discrete_sequence=px.colors.sequential.Plasma_r) 
    fig2.update_layout(showlegend=False)
    
    
    df_trend=df_use[df_use[category]==cat_value]   
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').mean().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[t1,t2,t3,t4,t5,t6,t7,t8,t9],
            var_name='Source Type',
            value_name='Value (kBtu)')   
    
    fig3 = px.line(df_trend, x='Year Ending', y='Value (kBtu)', markers=True,
                   color='Source Type',
                   title='Average Energy Usage Trend for 2016 - 2020 for '+category+' : ' +cat_value,
                  color_discrete_sequence=px.colors.sequential.Turbo)    
    fig3.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig3.for_each_trace(lambda t: t.update(name = use_dict[t.name],
                                      legendgroup = use_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, use_dict[t.name])
                                     )
                  )
    

    fig4 = px.sunburst(df_trend, path=['Year Ending', 'Source Type'], values='Value (kBtu)', color='Source Type',
                       color_discrete_sequence=px.colors.sequential.Turbo) 
    fig4.update_layout(showlegend=False)
    
    
    df_trend=df_use[df_use[category]==cat_value]   
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').mean().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[i1,i2,i3,i4,i5,i6,i7,i8,i9],
            var_name='Source Type',
            value_name='Value (kBtu/ft2)')   
    
    fig5 = px.line(df_trend, x='Year Ending', y='Value (kBtu/ft2)', markers=True,
                   color='Source Type',
                   title='Average Energy Use Intensity Trend for 2016 - 2020 for '+category+' : ' +cat_value,
                  color_discrete_sequence=px.colors.sequential.RdBu)    
    fig5.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig5.for_each_trace(lambda t: t.update(name = use_dict_intensity[t.name],
                                      legendgroup = use_dict_intensity[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, use_dict_intensity[t.name])
                                     )
                  )
    

    fig6 = px.sunburst(df_trend, path=['Year Ending', 'Source Type'], values='Value (kBtu/ft2)', color='Source Type',
                       color_discrete_sequence=px.colors.sequential.RdBu) 
    fig6.update_layout(showlegend=False)
     

    df1=df_use[df_use[category]==cat_value] 
    df1=df1[df1['Year Ending']==year][[i1,i2,i3,i4,i5,i6,i7,i8,i9,category]]

#     fig7 = px.scatter_matrix(df1,
#         dimensions=[i1,i2,i3,i4,i5,i6,i7,i8,i9],
# #         color="species", symbol="species",
#         title="Scatter matrix of Energy Use "+YearLabel +'('+category+':'+cat_value+')',
#         labels=use_dict_intensity) # remove underscore
#     fig7.update_traces(diagonal_visible=False)


#     fig7= ff.create_scatterplotmatrix(df1[num_cols], diag='box', index='index',colormap='Portland',
#                                   colormap_type='cat',
#                                   height=700, width=700)

    df1=df1[[i1,i2,i3,i4,i5,i6,i7,i8,i9]].rename(columns=use_dict_intensity)
    corr = df1.corr() # Generate correlation matrix
    corr=corr.rename(columns=use_dict_intensity)
    corr_text = np.around(corr, decimals=2)
    fig7 = go.Figure(data= go.Heatmap(z=corr,
                                     x=corr.index.values,
                                     y=corr.columns.values,
                                     colorscale='earth',
                                     text=corr_text
                                     )
                    )
    
    fig7.update_layout(title_text='<b>Correlation Matrix '+YearLabel+' <br> (energy use intensity (kBtu/ft2))<b>',
                      title_x=0.5,
                      titlefont={'size': 14},
                      width=550, height=550,
                      xaxis_showgrid=False,
                      yaxis_showgrid=False,
                      yaxis_autorange='reversed', 
                      paper_bgcolor=None
                      )
    return [fig1,fig2, fig3, fig4, fig5, fig6, fig7] 


## GREEN GAS EMISSION CONTENTS

@app.callback(
    Output('c_emission_property', 'children'),
    Output('t_emission_property', 'children'),
    Output('c_emission_total', 'children'),
    Output('t_emission_total', 'children'),
    Output('c_emission_avg', 'children'),
    Output('t_emission_avg', 'children'),
    Output('Category_text_emission','children'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list_emission', 'value')])
def update_cards(years, category, cat_value):
        
    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', category, 'Year Ending',
                                                       e1,e2,e3,e4,
                                                       s1,s2,s3,s4]]
    df=df[~df[s1].isna()]    
    df=df_energy[df_energy['Year Ending'].isin(years)]
    df=df[df[category]==cat_value]  
    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
    
    
    card1=str("{:,}".format(df[s1].count()))
    text1='Property Collected  '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    card2=str("{:,}".format(round(df[s1].sum(), 4))) + ' (kBtu)'
    text2='Total GHG Emission '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    card3=str("{:,}".format(round(df[s1].mean(), 4))) +' (kWh)'
    text3='Average GHG Emission '+YearLabel +' <br> ('+category+' : '+cat_value+')'
    
    category_name=category+' : '
    
    return [card1, text1, card2, text2, card3, text3, category_name]
        


@app.callback(
    Output('dropdown_category_emission', 'children'),
    [Input('Categories', 'value')])
def update_dropdown_category(category):
    value=df_energy[category].unique()[0]
    if category=='Community Board':
        value='103'
    elif category=='Borough':
        value='MANHATTAN'
    return dcc.Dropdown(df_energy[category].unique(), value=value,
                        id='category_list_emission')

@app.callback(
    Output('emissionmap', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list_emission', 'value')])
def update_map(years, category, cat_value):      
    df=df_energy[df_energy['Year Ending'].isin(years)][['Property Id', 'Property Name', 
                                                        category, 'Year Ending',
                                                        'Latitude','Longitude',
                                                       e1,e2,e3,e4,
                                                       s1,s2,s3,s4]]
    df=df[~df[s1].isna()]
    df=df[df[category]==cat_value]  
    lat=df['Latitude'].iloc[0]
    lon=df['Longitude'].iloc[0]
    fig1 = px.scatter_mapbox(
        df, 
        lat="Latitude", 
        lon="Longitude", 
        color=category, 
        opacity=0.6,
        height=600,
        hover_name='Property Name', 
        hover_data=[s1,s2,s3,s4],
                    size=s1,
                    size_max=15,
                    zoom=10)
    
    fig1.update_layout(
                    mapbox=dict(
                        bearing=0,
                        center=dict(
#                             lat=40.7729206,
#                             lon=-73.9824891
                            lat=lat,
                            lon=lon,
                        ),
                        pitch=0,
                        zoom=12))
    fig1.update_layout(mapbox_style="carto-positron")
    fig1.update_layout(showlegend=False)
    fig1.update_layout(title='GHG Emission Map for Manhattan')

    return fig1


@app.callback(
    Output('line_emission_yeartrend_total', 'figure'),
    Output('pie_emission_yeartrend_total', 'figure'),
    Output('line_emission_yeartrend_avg', 'figure'),
    Output('pie_emission_yeartrend_avg', 'figure'),
    Output('line_emission_yeartrend_mean', 'figure'),
    Output('pie_emission_yeartrend_mean', 'figure'),
    Output('heatmap_emission', 'figure'),
#     Output('bar_use_total', 'figure'),
#     Output('pie_use_total', 'figure'),
#     Output('bar_use_mean', 'figure'),
#     Output('pie_use_mean', 'figure'),
    [Input('Year', 'value'),
    Input('Categories', 'value'),
    Input('category_list_emission', 'value')])
def update_bar(years, category, cat_value):    
    year=years[0]
    YearLabel='for '+str(year)
    if year==2014:
        YearLabel='for 2016 - 2020 (mean)'
    elif year==2015:
        YearLabel='for 2016 - 2020 (total)'
        
    
    df_trend=df_emission[df_emission[category]==cat_value]  
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').sum().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[s1,s2,s3,s4],
            var_name='Emission Type',
            value_name='Value (Metric Tons CO2e)')   
    
    fig1 = px.line(df_trend, x='Year Ending', y='Value (Metric Tons CO2e)', markers=True,
                   color='Emission Type',
                   title='Total GHG Emission Trend for 2016 - 2020 for '+cat_value,
                  color_discrete_sequence=px.colors.sequential.Plasma_r)    
    fig1.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig1.for_each_trace(lambda t: t.update(name = emission_dict[t.name],
                                      legendgroup = emission_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, emission_dict[t.name])
                                     )
                  )
    

    fig2 = px.sunburst(df_trend, path=['Year Ending', 'Emission Type'], 
                       values='Value (Metric Tons CO2e)', 
                       color='Emission Type',
                       color_discrete_sequence=px.colors.sequential.Plasma_r) 
    fig2.update_layout(showlegend=False)
    
    
    df_trend=df_emission[df_emission[category]==cat_value]   
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').mean().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[s1,s2,s3,s4],
            var_name='Emission Type',
            value_name='Value (Metric Tons CO2e)')   
    
    fig3 = px.line(df_trend, x='Year Ending', y='Value (Metric Tons CO2e)', markers=True,
                   color='Emission Type',
                   title='Average GHG Emission Trend for 2016 - 2020 for '+category+' : ' +cat_value,
                  color_discrete_sequence=px.colors.sequential.Turbo)    
    fig3.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig3.for_each_trace(lambda t: t.update(name = emission_dict[t.name],
                                      legendgroup = emission_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, emission_dict[t.name])
                                     )
                  )
    

    fig4 = px.sunburst(df_trend, path=['Year Ending', 'Emission Type'], values='Value (Metric Tons CO2e)', 
                       color='Emission Type',
                       color_discrete_sequence=px.colors.sequential.Turbo) 
    fig4.update_layout(showlegend=False)
    
    
    df_trend=df_emission[df_emission[category]==cat_value]   
    df_trend=df_trend[df_trend['Year Ending'].isin([2016, 2017, 2018, 2019, 2020])]
    df_trend=df_trend.groupby('Year Ending').mean().reset_index()
    df_trend=pd.melt(df_trend, id_vars=['Year Ending'], 
            value_vars=[e1,e2,e3,e4],
            var_name='Emission Type',
            value_name='Value (kgCO2e/ft²)')   
    
    fig5 = px.line(df_trend, x='Year Ending', y='Value (kgCO2e/ft²)', markers=True,
                   color='Emission Type',
                   title='Average GHG Emission Intensity Trend for 2016 - 2020 for '+category+' : ' +cat_value,
                  color_discrete_sequence=px.colors.sequential.RdBu)    
    fig5.update_xaxes(tickmode = 'array',
                 tickvals = [2016, 2017, 2018, 2019, 2020],
#                  ticktext= [2016, 2017, 2018, 2019, 2020]
                     )
    fig5.for_each_trace(lambda t: t.update(name = emission_dict_intensity[t.name],
                                      legendgroup = emission_dict_intensity[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, emission_dict_intensity[t.name])
                                     )
                  )
    

    fig6 = px.sunburst(df_trend, path=['Year Ending', 'Emission Type'], values='Value (kgCO2e/ft²)', color='Emission Type',
                       color_discrete_sequence=px.colors.sequential.RdBu) 
    fig6.update_layout(showlegend=False)
     

    df1=df_emission[df_emission[category]==cat_value] 
    df1=df1[df1['Year Ending']==year][[e1,e2,e3,e4,category]]

    df1=df1[[e1,e2,e3,e4]].rename(columns=emission_dict_intensity)
    corr = df1.corr() # Generate correlation matrix
    corr=corr.rename(columns=emission_dict_intensity)
    corr_text = np.around(corr, decimals=2)
    fig7 = go.Figure(data= go.Heatmap(z=corr,
                                     x=corr.index.values,
                                     y=corr.columns.values,
                                     colorscale='earth',
                                     text=corr_text
                                     )
                    )
    
    fig7.update_layout(title_text='<b>Correlation Matrix '+YearLabel+' <br> (GHG emission intensity (kgCO2e/ft2))<b>',
                      title_x=0.5,
                      titlefont={'size': 14},
                      width=550, height=550,
                      xaxis_showgrid=False,
                      yaxis_showgrid=False,
                      yaxis_autorange='reversed', 
                      paper_bgcolor=None
                      )
    return [fig1,fig2, fig3, fig4, fig5, fig6, fig7] 

app.run_server(mode='external', port=8051)

Dash app running on http://127.0.0.1:8051/


# Other evidence

NYC Clean Heat Dataset (Historical)

https://data.cityofnewyork.us/City-Government/NYC-Clean-Heat-Dataset-Historical-/8isn-pgv3

CATS Permits

https://data.cityofnewyork.us/Environment/CATS-Permits/f4rp-2kvy

In [19]:
df_cat=pd.read_csv('data/CATS_Permits.csv')
df_clean=pd.read_csv('data/NYC_Clean_Heat_Dataset__Historical_.csv')

In [20]:
df_cat.head()

,RequestID,ApplicationID,RequestType,House,Street,Borough,Bin,Block,Lot,OwnerName,ExpirationDate,Make,Model,BurnerMake,BurnerModel,PrimaryFuel,SecondaryFuel,Quantity,IssueDate,status,PremiseName
0,62416,CA205697,REGISTRATION,5403,13 AVENUE,BROOKLYN,3139408.000,5677.000,9.000,BARNET CLEANERS INC.,11/18/2000,"FULTON, FB-015-A","FULTON, FB-015-A","FULTON, INTEGRAL","FULTON, INTEGRAL",NO2FUEL,NONE,1.000,11/03/1997 12:00:00 AM,EXPIRED,NaN
1,108962,CA205698,CERTIFICATE TO OPERATE,2185,BATHGATE AVENUE,BRONX,2011538.000,3049.000,37.000,NYC SCH.CONSTR.AUYHORITY,03/23/2002,(2)FEDERAL (NEW) PLW-165,(2)FEDERAL (NEW) PLW-165,GORDON PIATT FL12.1-GO 50,GORDON PIATT FL12.1-GO 50,NATURALGAS,NONE,2.000,10/07/1998 12:00:00 AM,EXPIRED,P.S.59
2,131634,CA205752,CERTIFICATE TO OPERATE,710,EAST 138 STREET,BRONX,2003792.000,2566.000,18.000,D.M.T.REALTY CORP OF NY,01/19/1995,PACIFIC FT,PACIFIC FT,HEV-E-OIL AMDH3,HEV-E-OIL AMDH3,NO4FUEL,NONE,1.000,12/02/1986 12:00:00 AM,CANCELLED,NaN
3,367122,CA205773,CERTIFICATE TO OPERATE,1000,PELHAM PARKWAY,BRONX,2049614.000,4329.000,1.000,MORNINGSIDE NURSING AND REEHABILITATION CENTER,03/06/2019,CLEAVER BROOKS,CB200-200,CLEAVER BROOKS,CB200-200,NATURALGAS,NO2FUEL,1.000,06/27/2022 09:07:15 AM,CANCELLED,MORNINGSIDE HOUSE NURSING
4,293138,CA205786,CERTIFICATE TO OPERATE,930,SHERIDAN AVENUE,BRONX,2002706.000,2454.000,19.000,"SHERIDAN ONE COMPANY, LLC",10/17/2022,FEDERAL,FST-175,INDUSTRIAL COMBUSTION,DEG-84P,NATURALGAS,NO4FUEL,1.000,08/27/2020 03:44:55 PM,CURRENT,SHERIDAN ONE COMPANY


In [21]:
df_cat['PrimaryFuel'].unique()

array(['NO2FUEL', 'NATURALGAS', 'NO4FUEL', 'NO6FUEL', 'NONE', 'NO2FUELB2',
       nan, 'OTHER', 'OILORDIESEL', 'NATURALGASOTHER',
       'NATURALGASNO2FUEL'], dtype=object)

In [22]:
df_clean.head()

,"Borough, Block, Lot #",Street Address,Postcode,Borough,Utility,Building Manager,Owner,Owner Address,Owner Telephone,DEP Boiler Application #,#6 Deadline,Boiler Model,# of Identical Boilers,Boiler Capacity (Gross BTU),Boiler Installation Date,Boiler Age Range,Est. Retirement Year,Burner Model,Primary Fuel,Total Gallons (High),Total Gallons (Low),Total MMBTU (High),Total MMBTU (low),Greener Greater Buildings,GGB Deadline,Building Type,Council District,Community Board,Bldg Sqft,# of Bldgs,# of Floors,# of Res. Units,Total Units,Year Built,Condo?,Coop?,Latitude,Longitude,Census Tract,BIN,BBL,NTA
0,1008120001,155 WEST 36 STREET,10018.000,Manhattan,Con Edison,485 7 AVE.ASSOC./COLLIERS,485 SHUR LLC,"485 7 AVENUE#777, MANHATTAN NY 10018",212-971-4000,CA160181H,2012,FEDERAL FST-200,2.000,8.400,1981.000,26 to 30 years old,2016.000,ICI #DEG 84P,#6,131093,91765,13429.080,19184.400,1.000,2022.000,Office Buildings,3.000,M 05,"* 235,450",1.000,15.000,0.000,70.000,1906.000,NaN,NaN,40.752,-73.989,109.000,1015235.000,1008127501.000,Midtown-Midtown South
1,1008340048,330 5 AVENUE,10001.000,Manhattan,Con Edison,SKYLER 330 LLCC/O SHULSKY PROPERTIES INC.,SHULSKY PROPERTIES INC.,"307 FITH AVE, NY NY 10016",212 984-8370,CA323565K,2015,PACIFIC #9303 A,1.000,0.000,1965.000,41 - 45 years old,2010.000,PETRO WD7AH,#4,114975,80483,5075.000,7250.000,1.000,2014.000,Office Buildings,3.000,M 05,"* 145,000",1.000,14.000,0.000,62.000,1926.000,NaN,NaN,40.748,-73.985,76.000,1015853.000,1008340048.000,Midtown-Midtown South
2,1008390009,49 WEST 37 STREET,10018.000,Manhattan,Con Edison,49 W 37 ST REALTY CO,49 W 37TH ST REALTY CO,"440 PARK AVENUE SOUTH, MANHATTAN NY 10016",212 685-6400,CA145582N,0,FEDERAL FST 200,1.000,8.400,1982.000,26 to 30 years old,2017.000,JOHNSON FD68CAHM-200,#4,44640,31248,4687.200,6696.000,1.000,2019.000,Loft Buildings,3.000,M 05,"* 140,898",1.000,18.000,0.000,24.000,1925.000,NaN,NaN,40.751,-73.985,84.000,1015958.000,1008390009.000,Midtown-Midtown South
3,1008670001,411 5 AVENUE,10016.000,Manhattan,Con Edison,ADMS& CO. REAL ESTATE,ADAMS & CO. LLC/FRED LIGUORI,"411 5 AVENUE, MANHATTAN NY 10016",212-679-5500,CA417870Y,0,FEDERAL FST-175,1.000,5.500,1970.000,36 - 40 year old,2010.000,ACE FDL04 SIZE 200,#4,100667,70467,4725.000,6750.000,1.000,2017.000,Office Buildings,3.000,M 05,"* 116,598",1.000,11.000,0.000,1.000,1915.000,NaN,NaN,40.750,-73.983,82.000,1017191.000,1008670001.000,Murray Hill-Kips Bay
4,1022420029,639 WEST 207 STREET,10034.000,Manhattan,Con Edison,WEINER REALTY,PINNACLE WASHINGTON HEIGHTS LLC,"P.O.BO. 1920, NEW YORK NY 10116",NaN,CA068682Y,0,ROCKMILLS MP80,1.000,3.300,1982.000,26 to 30 years old,2017.000,ICI MEG34P,#4,40004,28003,4200.000,6000.000,0.000,NaN,Walk-Up Apartments,7.000,M 12,"* 37,365",1.000,5.000,56.000,58.000,1925.000,NaN,NaN,40.869,-73.922,303.000,1064990.000,1022420029.000,Marble Hill-Inwood


In [23]:
df_clean['Primary Fuel'].unique()

array(['#6', '#4'], dtype=object)

In [24]:
df_cat[df_cat['PrimaryFuel']=='OTHER'].shape

(5, 21)